# EE cloud optimized geotiff example
This notebook shows an example of how to upload a cloud optimized geotiff to GEE and use it both as an asset and directly.

In [ ]:
import pathlib


Here we use the gebco dataset as an example

In [ ]:
path = pathlib.Path('/Users/baart_f/data/gebco')
tiffs = list(path.glob('*.tif'))

Here we convert the data to a cloud optimized geotiff with .co in the name.

In [ ]:
for f in tiffs:
    # co  -> cloud optimized
    new_name = f.with_name(f.stem + '.co').with_suffix('.tiff')
    !rio cogeo create {f} {new_name}

Now you have to upload the files that are created to a bucket. You can use gsutil for that. I did it by hand. 

In [ ]:
# upload data to bucket manually
!ls /Users/baart_f/data/gebco/


We need to login to the GEE. This requires a file with credentials.

In [ ]:
# This has details about the Earth Engine Python Authenticator client.
from ee import oauth
from google_auth_oauthlib.flow import Flow
import json

# Build the `client_secrets.json` file by borrowing the
# Earth Engine python authenticator.
client_secrets = {
    'web': {
        'client_id': oauth.CLIENT_ID,
        'client_secret': oauth.CLIENT_SECRET,
        'redirect_uris': [oauth.REDIRECT_URI],
        'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
        'token_uri': 'https://accounts.google.com/o/oauth2/token'
    }
}

# Write to a json file.
client_secrets_file = 'client_secrets.json'
with open(client_secrets_file, 'w') as f:
  json.dump(client_secrets, f, indent=2)

# Start the flow using the client_secrets.json file.
flow = Flow.from_client_secrets_file(client_secrets_file,
    scopes=oauth.SCOPES,
    redirect_uri=oauth.REDIRECT_URI)

# Get the authorization URL from the flow.
auth_url, _ = flow.authorization_url(prompt='consent')

# Print instructions to go to the authorization URL.
oauth._display_auth_instructions_with_print(auth_url)
print('\n')

# The user will get an authorization code.
# This code is used to get the access token.
code = input('Enter the authorization code: \n')
flow.fetch_token(code=code)

# Get an authorized session from the flow.
session = flow.authorized_session()

We need a list of all the files that we want to process.

In [ ]:
names = [
    "gebco_2020_n0.0_s-90.0_w-90.0_e0.0.tiff",
    "gebco_2020_n0.0_s-90.0_w-180.0_e-90.0.tiff",
    "gebco_2020_n0.0_s-90.0_w0.0_e90.0.tiff",
    "gebco_2020_n0.0_s-90.0_w90.0_e180.0.tiff",
    "gebco_2020_n90.0_s0.0_w-90.0_e0.0.tiff",
    "gebco_2020_n90.0_s0.0_w-180.0_e-90.0.tiff",
    "gebco_2020_n90.0_s0.0_w0.0_e90.0.tiff",
    "gebco_2020_n90.0_s0.0_w90.0_e180.0.tiff"
]

name.rstrip('.tiff').replace('.', '_')

This shows how to upload the assets. 

In [ ]:
# Where Earth Engine assets are kept.
project_folder = 'earthengine-legacy'
# Your user folder name and new asset name.
asset_id = 'users/fbaart/gebco/cog'


for name in names:
    # Request body as a dictionary.
    request = {
      'type': 'IMAGE',
      'cloudStorageLocation': {
        'uris': [f'gs://ee-cog-test/cog/{name}']
      },
      'properties': {
        'source': 'https://github.com/openearth/dgds-backend'
      },
      'startTime': '2016-01-01T00:00:00.000000000Z',
      'endTime': '2016-12-31T15:01:23.000000000Z',
    }

    from pprint import pprint
    pprint(json.dumps(request))

    ee_name = name.rstrip('.tiff').replace('.', '_')
    
    url = f'https://earthengine.googleapis.com/v1alpha/projects/{project_folder}/assets?assetId={asset_id}/{ee_name}'

    response = session.post(
      url = url,
      data = json.dumps(request)
    )

    pprint(json.loads(response.content))

This uploads them in the traditional way.

In [ ]:
names = [
    "gebco_2020_n0.0_s-90.0_w-90.0_e0.0.tif",
    "gebco_2020_n0.0_s-90.0_w-180.0_e-90.0.tif",
    "gebco_2020_n0.0_s-90.0_w0.0_e90.0.tif",
    "gebco_2020_n0.0_s-90.0_w90.0_e180.0.tif",
    "gebco_2020_n90.0_s0.0_w-90.0_e0.0.tif",
    "gebco_2020_n90.0_s0.0_w-180.0_e-90.0.tif",
    "gebco_2020_n90.0_s0.0_w0.0_e90.0.tif",
    "gebco_2020_n90.0_s0.0_w90.0_e180.0.tif"
]

for name in names:
    asset_id = 'users/fbaart/gebco/gebco-native'
    gs_name = f'gs://ee-cog-test/gebco/{name}'
    ee_name = name.rstrip('.tiff').replace('.', '_')
    !earthengine upload image --asset_id {asset_id}/{ee_name} {gs_name}

The result is two assets gebco-native and gebco-cog. These can now be used for benchmarks.